In [13]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

# Canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Default pen settings
pen_color = (255, 0, 0)
pen_size = 5

# Smoothing buffer
smooth_buffer = deque(maxlen=5)  # average last 5 points

prev_x, prev_y = None, None

def smooth_point(x, y):
    smooth_buffer.append((x, y))
    avg_x = int(np.mean([p[0] for p in smooth_buffer]))
    avg_y = int(np.mean([p[1] for p in smooth_buffer]))
    return avg_x, avg_y

cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
) as hands:

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (640, 480))
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        output = hands.process(rgb)

        if output.multi_hand_landmarks:
            hand = output.multi_hand_landmarks[0]
            lm = hand.landmark

            # Index finger tip
            ix, iy = int(lm[8].x * 640), int(lm[8].y * 480)

            # Smooth it
            sx, sy = smooth_point(ix, iy)

            # DRAW THE SMOOTH POINT
            cv2.circle(frame, (sx, sy), 8, (0, 0, 255), -1)

            # Distance threshold for smoothness
            if prev_x is not None and prev_y is not None:
                dist = np.hypot(sx - prev_x, sy - prev_y)

                if dist > 3:  # ignore tiny shakes
                    cv2.line(canvas, (prev_x, prev_y), (sx, sy), pen_color, pen_size)

            prev_x, prev_y = sx, sy

        else:
            prev_x, prev_y = None, None
            smooth_buffer.clear()

        combined = cv2.addWeighted(frame, 0.7, canvas, 0.7, 0)
        cv2.imshow("SUPER SMOOTH Finger Drawing", combined)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            canvas = np.zeros((480, 640, 3), dtype=np.uint8)
        if key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
